

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CONTEXTUAL_SPELL_CHECKER.ipynb)




# **Spell checking for clinical documents**

To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

In [1]:
import os
import json

with open('/content/workshop_license_keys.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['secret']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['JSL_OCR_LICENSE'] = license_keys['JSL_OCR_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']

Install dependencies

In [ ]:
# Install Java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark and SparkNLP
! pip install --ignore-installed -q pyspark==2.4.4
! python -m pip install --upgrade spark-nlp-jsl==2.5.2 --extra-index-url https://pypi.johnsnowlabs.com/$secret
! pip install --ignore-installed -q spark-nlp==2.5.2

Import dependencies into Python and start the Spark session

In [3]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']
import sparknlp

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import pyspark.sql.functions as F

builder = SparkSession.builder \
    .appName('Spark NLP Licensed') \
    .master('local[*]') \
    .config('spark.driver.memory', '16G') \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.kryoserializer.buffer.max', '2000M') \
    .config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.2') \
    .config('spark.jars', f'https://pypi.johnsnowlabs.com/{secret}/spark-nlp-jsl-2.5.2.jar')
    
spark = builder.getOrCreate()

## 2. Select the NER model and construct the pipeline

In [10]:
document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

tokenizer = RecursiveTokenizer()\
  .setInputCols(["document"])\
  .setOutputCol("token")\
  .setPrefixes(["\"", "(", "[", "\n"])\
  .setSuffixes([".", ",", "?", ")","!", "‘s"])

spell_model = ContextSpellCheckerModel.pretrained('spellcheck_clinical', "en", "clinical/models")\
  .setInputCols("token")\
  .setOutputCol("corrected")

finisher = Finisher().setInputCols("corrected")

light_pipeline = Pipeline(stages = [
                                    document_assembler,
                                    tokenizer,
                                    spell_model,
                                    finisher
                                    ])
## For comparison
full_pipeline = Pipeline(
    stages = [
              document_assembler,
              tokenizer,
              spell_model
  ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")
pipeline_model = full_pipeline.fit(empty_ds)
l_pipeline_model = LightPipeline(light_pipeline.fit(empty_ds))

spellcheck_clinical download started this may take some time.
Approximate size to download 145 MB
[OK!]


## 3. Create example inputs

In [11]:
# Enter examples as strings in this array
input_list = ["The pateint is a 5-mont-old infnt who presented initially on Monday with a cold, cugh, and runny nse for 2 days. Mom states she had no fevr. Her appetite was good but she was spitting up a lot. She had no difficulty breathin and her cough was described as dry and hacky. At that time, pysicl exam showed a right TM, which was red. Left TM was okay. She was fairly congsted but looked happy and playful. She was started on Amxil and Aldx and we told to recheck in 2 weaks to recheck her ear. Mom returned to clinic again today because she got much worse ovrnght. She was having dificlty breathing. She was much more congested and her apetit had decrsed significantly today. She also spked a tempratre yesterday of 102.6 and always hvng trouble sleping scondry to congestion."]

## 4. Use the pipeline to create outputs

Full Pipeline

In [12]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)

Light Pipeline

In [14]:
# Light pipelines expect a single example.
light_result = l_pipeline_model.annotate(input_list[0])

## 5. Visualize results

Visualize comparison as dataframe

In [13]:
exploded = F.explode(F.arrays_zip('token.result', 'corrected.result'))
select_expression_0 = F.expr("cols['0']").alias("original")
select_expression_1 = F.expr("cols['1']").alias("corrected")
result.select(exploded.alias("cols")) \
    .select(select_expression_0, select_expression_1).show(truncate=False)

+----------+-----------+
|original  |corrected  |
+----------+-----------+
|The       |The        |
|pateint   |patient    |
|is        |is         |
|a         |a          |
|5-mont-old|5-month-old|
|infnt     |infant     |
|who       |who        |
|presented |presented  |
|initially |initially  |
|on        |on         |
|Monday    |Monday     |
|with      |with       |
|a         |a          |
|cold      |cold       |
|,         |,          |
|cugh      |cough      |
|,         |,          |
|and       |and        |
|runny     |runny      |
|nse       |nose       |
+----------+-----------+
only showing top 20 rows



Vizualise light pipeline and finished result

In [16]:
# this finished result does not need parsing and can directly be used an any other task.
light_result['corrected']

['The',
 'patient',
 'is',
 'a',
 '5-month-old',
 'infant',
 'who',
 'presented',
 'initially',
 'on',
 'Monday',
 'with',
 'a',
 'cold',
 ',',
 'cough',
 ',',
 'and',
 'runny',
 'nose',
 'for',
 '2',
 'days',
 '.',
 'Mm',
 'states',
 'she',
 'had',
 'no',
 'fever',
 '.',
 'Her',
 'appetite',
 'was',
 'good',
 'but',
 'she',
 'was',
 'spitting',
 'up',
 'a',
 'lot',
 '.',
 'She',
 'had',
 'no',
 'difficulty',
 'breathing',
 'and',
 'her',
 'cough',
 'was',
 'described',
 'as',
 'dry',
 'and',
 'back',
 '.',
 'At',
 'that',
 'time',
 ',',
 'physical',
 'exam',
 'showed',
 'a',
 'right',
 'TM',
 ',',
 'which',
 'was',
 'red',
 '.',
 'Left',
 'TM',
 'was',
 'okay',
 '.',
 'She',
 'was',
 'fairly',
 'congested',
 'but',
 'looked',
 'happy',
 'and',
 'painful',
 '.',
 'She',
 'was',
 'started',
 'on',
 'Amoxil',
 'and',
 'Aldex',
 'and',
 'we',
 'told',
 'to',
 'recheck',
 'in',
 '2',
 'weeks',
 'to',
 'recheck',
 'her',
 'ear',
 '.',
 'Mm',
 'returned',
 'to',
 'clinic',
 'again',
 'today'